In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


In [2]:

# URL of the website
url = "https://m.football-lineups.com/match/503709/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Check if the request was successful
if response.status_code == 200:
    # Save the HTML content to a file
    with open("match_data.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("HTML saved successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


HTML saved successfully.


In [3]:
from bs4 import BeautifulSoup
import re
import pandas as pd


# URL of the website
# url = "https://m.football-lineups.com/match/503709/"
url = 'https://m.football-lineups.com/match/503704/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Check if the request was successful
if response.status_code == 200:
    # Save the HTML content to a file
    with open("match_data.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("HTML saved successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


# Extract match date
match_date = None
match_date_div = soup.find("div", class_="inner", style="text-align:center")
if match_date_div:
    date_search = re.search(r"\d{4}-\d{2}-\d{2}", match_date_div.text)
    if date_search:
        match_date = date_search.group()

# Extract referee name and link
referee_name = None
referee_link = None
referee_tag = soup.find("a", href=re.compile(r"/referee/"))
if referee_tag:
    referee_name = referee_tag.text.strip()
    referee_link = referee_tag['href'].replace('https://','')

# Extract lineups of both teams with player links
lineups = {"Team": [], "Players": [], "Player Link": []}
lineup_tables = soup.find_all("table", class_="table table-responsive table-condensed table-hover")
for lineup_table in lineup_tables:
    team_header = lineup_table.find_previous("div", class_="color-fondo")
    if team_header:
        team_name = team_header.text.strip()
        players_info = lineup_table.find_all("a", href=re.compile(r"/footballer/"))
        for player in players_info:
            player_name = player.text.strip()
            player_link = player['href'].replace('https://','')
            lineups["Team"].append(team_name)
            lineups["Players"].append(player_name)
            lineups["Player Link"].append(player_link)

# Create DataFrames
match_info_data = {
    "Match Date": [match_date],
    "Referee Name": [referee_name],
    "Referee Link": [referee_link]
}

match_info_df = pd.DataFrame(match_info_data)
lineups_df = pd.DataFrame(lineups)

# Display DataFrames
print("Match Information DataFrame:")
print(match_info_df)
print("\nTeam Lineups DataFrame with Player Links:")
print(lineups_df)


HTML saved successfully.
Match Information DataFrame:
   Match Date Referee Name               Referee Link
0  2024-11-23       Madley  /referee/2325/Andy-Madley

Team Lineups DataFrame with Player Links:
              Team      Players          Player Link
0   Leicester City    Hermansen  /footballer/132159/
1   Leicester City       Justin  /footballer/114670/
2   Leicester City         Faes  /footballer/100378/
3   Leicester City        Okoli  /footballer/133819/
4   Leicester City  Kristiansen  /footballer/144690/
5   Leicester City      McAteer  /footballer/156265/
6   Leicester City        Ndidi   /footballer/80418/
7   Leicester City      Soumaré  /footballer/106434/
8   Leicester City        Winks   /footballer/88884/
9   Leicester City  E Khannouss  /footballer/179376/
10  Leicester City        Vardy   /footballer/63664/
11         Chelsea    R Sanchez  /footballer/114744/
12         Chelsea       Fofana  /footballer/137035/
13         Chelsea   Badiashile  /footballer/121586/
